In [1]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [2]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Assuming 'data' and 'target' are 1D arrays
data = np.array(data).reshape(-1, 1)
target = np.array(target).reshape(-1, 1)

data_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = data_scaler.fit_transform(data)

target_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_target = target_scaler.fit_transform(target)


ValueError: Expected 2D array, got 1D array instead:
array=[246.48001099 247.09999084 248.44999695 ...  93.19999695  93.95999908
  94.06999969].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [30]:
import joblib

joblib.dump(data_scaler,'data_scaler.sav')
joblib.dump(target_scaler,'target_scaler.sav')

['target_scaler.sav']

In [31]:
scaled_data=scaled_data.reshape(scaled_data.shape[0],scaled_data.shape[1],1)
scaled_target=scaled_target.reshape(scaled_target.shape[0],scaled_target.shape[1],1)

In [32]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(scaled_data,scaled_target,test_size=0.2)

In [33]:
from keras.layers import Input,Dense,Conv1D,MaxPool1D,UpSampling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer=Input(shape=scaled_data.shape[1:])  

model=Conv1D(32,(3),activation='relu',padding='same')(input_layer)
model=MaxPool1D((2),padding='same')(model)
model=Conv1D(32,(3),activation='relu',padding='same')(model)
encoder=MaxPool1D((2),padding='same')(model)

model=Conv1D(32,(3),activation='relu',padding='same')(encoder)
model=UpSampling1D((2))(model) 
model=Conv1D(32,(3),activation='relu',padding='same')(model)
model=UpSampling1D((2))(model) 

decoder=Conv1D(1,(3),activation='relu',padding='same')(model)

autoencoder=Model(input_layer,decoder)

autoencoder.compile(optimizer='adam',loss='mse')

In [34]:
checkpoint = ModelCheckpoint('models/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
autoencoder.fit(train_data,train_target,epochs=100,validation_data=(test_data, test_target),callbacks=[checkpoint])

Epoch 1/100


StagingError: in user code:

    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\Programs\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = model

In [35]:
np.save('test_data',test_data)
np.save('test_target',test_target)